In [1]:
import pandas as pd
from typing import List, Dict, Any
import numpy as np

In [2]:
users_df = pd.read_csv("C:\\Users\\gjric\\Desktop\\Valorant Analysis\\data\\users_dim.csv", parse_dates=["join_date"])
agents_df = pd.read_csv("C:\\Users\\gjric\\Desktop\\Valorant Analysis\\data\\agents_dim.csv")
maps_df = pd.read_csv("C:\\Users\\gjric\\Desktop\\Valorant Analysis\\data\\maps_dim.csv")


In [3]:
def base_params(
    users_df: pd.DataFrame,
    agents_df: pd.DataFrame,
    maps_df: pd.DataFrame,
    per_day_match_counter: int = 2,
    start_date: str = "2025-01-01",
    end_date: str = "today",
) -> pd.DataFrame:
    """
    Create a base dataframe of matches:
      - date range from start_date to end_date (daily)
      - per_day_match_counter matches per day
      - each match: 10 unique users, 10 unique agents, 1 map, sides assigned
    """

    # Date handling
    start_dt = pd.to_datetime(start_date)
    end_dt = pd.to_datetime(end_date).normalize()
    all_rows = []
    match_seq = 1  # incremental match id

    # Pre-filter playable agents
    if "isPlayable" in agents_df.columns:
        playable_agents = agents_df[agents_df["isPlayable"] == True].copy()
    else:
        playable_agents = agents_df.copy()

    N_PLAYERS = 10

    for date in pd.date_range(start=start_dt, end=end_dt):

        # Only players who have joined on or before this date
        eligible_users = users_df[users_df["join_date"] <= date]
        if len(eligible_users) < N_PLAYERS:
            # Not enough players yet, skip this date
            continue

        for _ in range(per_day_match_counter):
            # ----------------------------
            # 1. Sample 10 players
            # ----------------------------
            match_players = eligible_users.sample(
                n=N_PLAYERS, replace=False
            ).reset_index(drop=True)

            # ----------------------------
            # 2. Sample 10 agents
            # ----------------------------
            if len(playable_agents) < N_PLAYERS:
                raise ValueError("Not enough playable agents to assign uniquely.")

            agents_sample = playable_agents.sample(
                n=N_PLAYERS, replace=False
            ).reset_index(drop=True)

            # Use your actual columns: adjust if they differ
            # Common API columns: uuid, displayName
            match_players["agent_id"]   = agents_sample["uuid"]
            match_players["agent_name"] = agents_sample.get(
                "displayName", agents_sample.get("name", "")
            )

            # ----------------------------
            # 3. Sample a map for the match
            # ----------------------------
            if maps_df.empty:
                raise ValueError("maps_df is empty.")

            match_map = maps_df.sample(n=1).reset_index(drop=True)
            match_players["map_id"]   = match_map.loc[0, "uuid"]
            match_players["map_name"] = match_map.loc[0, "name"]

            """# ----------------------------
            # 4. Assign sides: 5 attackers, 5 defenders
            # ----------------------------
            attack_indices = np.random.choice(
                match_players.index, size=5, replace=False
            )
            match_players["is_attacker"] = 0
            match_players.loc[attack_indices, "is_attacker"] = 1
            match_players["is_defender"] = 1 - match_players["is_attacker"]"""

            # ----------------------------
            # 5. Add match metadata
            # ----------------------------
            match_id = f"MATCH_{match_seq:06d}"
            match_players["match_id"]   = match_id
            match_players["match_date"] = date.normalize()

            all_rows.append(match_players)
            match_seq += 1

    if not all_rows:
        return pd.DataFrame()  # nothing generated

    base_matches_df = pd.concat(all_rows, ignore_index=True)

    # Optional: enforce column order
    cols_order = [
        "match_id",
        "match_date",
        "user_id",
        "agent_id",
        "agent_name",
        "map_id",
        "map_name",
        "join_date",   # from users_df
        # other user columns (username, etc.) will follow
    ]
    base_matches_df = base_matches_df[
        [c for c in cols_order if c in base_matches_df.columns] +
        [c for c in base_matches_df.columns if c not in cols_order]
    ]

    match_df = pd.DataFrame()
    for m_id in base_matches_df["match_id"].unique():
        match_data = base_matches_df[base_matches_df["match_id"] == m_id]
        match_df = pd.concat([match_df, team_division(match_data)], ignore_index=True)


    return match_df

def team_division (
        match_df: pd.DataFrame,
        rounds_per_match: int = 25)-> pd.DataFrame:
    """
    For each match in match_df, generate round details:
        - rounds_per_match rounds
        - for each round: winning side (attackers/defenders)
    """

    teamside = np.random.choice(
        match_df.index, size=5, replace=False
        )
    match_df["team A"] = 0
    match_df.loc[teamside, "team A"] = 1
    match_df["team B"] = 1 - match_df["team A"]

    """match_df = generating_full_match_details_per_round(
        match_df = match_df, agents_df=agents_df
    )"""
    return match_df

In [4]:
df = base_params(
        users_df,
        agents_df,
        maps_df
    )
df

,match_id,match_date,user_id,agent_id,agent_name,map_id,map_name,join_date,username,tagline,rank_tier_uuid,team A,team B
0,MATCH_000001,2025-01-05,840,efba5359-4016-a1e5-7626-b1ae76895940,Vyse,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0840,#4199,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1
1,MATCH_000001,2025-01-05,814,0e38b510-41a8-5780-5e8f-568b2a4f2d6c,Iso,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-02,player0814,#2999,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1
2,MATCH_000001,2025-01-05,215,95b78ed7-4637-86d9-7e41-71ba8c293152,Harbor,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0215,#7345,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1
3,MATCH_000001,2025-01-05,101,add6443a-41bd-e414-f6ad-e58d267f4e95,Jett,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-01,player0101,#6371,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0
4,MATCH_000001,2025-01-05,920,1e58de9c-4950-5125-93e9-a0aee9f98746,Killjoy,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-01,player0920,#5618,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7915,MATCH_000792,2026-02-04,755,6f2a04ca-43e0-be17-7f36-b3908627744d,Skye,e2ad5c54-4114-a870-9641-8ea21279579a,Icebox,2025-04-28,player0755,#0104,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1
7916,MATCH_000792,2026-02-04,568,dade69b4-4f5a-8528-247b-219e5a1facd6,Fade,e2ad5c54-4114-a870-9641-8ea21279579a,Icebox,2025-08-24,player0568,#4366,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1
7917,MATCH_000792,2026-02-04,408,5f8d3a7f-467b-97f3-062c-13acf203c006,Breach,e2ad5c54-4114-a870-9641-8ea21279579a,Icebox,2025-12-07,player0408,#1969,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0
7918,MATCH_000792,2026-02-04,961,320b2a48-4d9b-a075-30f1-1f93a9b638fa,Sova,e2ad5c54-4114-a870-9641-8ea21279579a,Icebox,2025-01-14,player0961,#3051,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0


In [5]:
df[(df["match_id"]=="MATCH_000001")]

,match_id,match_date,user_id,agent_id,agent_name,map_id,map_name,join_date,username,tagline,rank_tier_uuid,team A,team B
0,MATCH_000001,2025-01-05,840,efba5359-4016-a1e5-7626-b1ae76895940,Vyse,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0840,#4199,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1
1,MATCH_000001,2025-01-05,814,0e38b510-41a8-5780-5e8f-568b2a4f2d6c,Iso,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-02,player0814,#2999,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1
2,MATCH_000001,2025-01-05,215,95b78ed7-4637-86d9-7e41-71ba8c293152,Harbor,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0215,#7345,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1
3,MATCH_000001,2025-01-05,101,add6443a-41bd-e414-f6ad-e58d267f4e95,Jett,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-01,player0101,#6371,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0
4,MATCH_000001,2025-01-05,920,1e58de9c-4950-5125-93e9-a0aee9f98746,Killjoy,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-01,player0920,#5618,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0
5,MATCH_000001,2025-01-05,258,a3bfb853-43b2-7238-a4f1-ad90e9e46bcc,Reyna,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0258,#0756,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0
6,MATCH_000001,2025-01-05,116,7f94d92c-4234-0a36-9646-3a87eb8b5c89,Yoru,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0116,#1207,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0
7,MATCH_000001,2025-01-05,948,6f2a04ca-43e0-be17-7f36-b3908627744d,Skye,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0948,#0983,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1
8,MATCH_000001,2025-01-05,594,601dbbe7-43ce-be57-2a40-4abd24953621,KAY/O,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0594,#9892,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0
9,MATCH_000001,2025-01-05,100,320b2a48-4d9b-a075-30f1-1f93a9b638fa,Sova,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-02,player0100,#0874,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1


In [6]:
def generating_full_match_details_per_round(
        match_df: pd.DataFrame,
        agents_df: pd.DataFrame,
        total_rounds: int = 25,
        first_round_credit: int = 800,
        attacker_round_wins: int = 0,
        defender_round_wins: int = 0,
        match_duration: int = np.random.randint(1500, 2400) # Match duration between 25 to 40 minutes
        )-> pd.DataFrame:
    """
    For each match in match_df, generate full round details:
        - rounds_per_match rounds
        - for each round: winning side (attackers/defenders)
    """
    match_records = []
    match_round_records = []
    for i in range (1, total_rounds + 1):
        round_id = f"{match_df['match_id'].iloc[0]}-R{i:02d}"

        if i == 1 or i==13:
            match_df = team_side_assignment(
                match_df,
                round_number=i
                )
        
        for row in match_df.itertuples(index=False):
            row_dict = dict(zip(match_df.columns, row))
            row_dict["round_id"] = round_id
            match_records.append(row_dict)
    return  pd.DataFrame(match_records)
    """round_details, team_spike_planted, team_spike_diffused, attacker_round_win, defender_round_win = events_per_round(round_df=pd.DataFrame(match_records)[pd.DataFrame(match_records)["round_id"]==round_id])
        match_round_records.extend(round_details)
        attacker_round_wins+= attacker_round_win
        defender_round_wins+= defender_round_win


        if attacker_round_wins == 13 or defender_round_wins == 13:
            break

        match_records_df = pd.DataFrame(match_records)
        match_records = match_records_df.groupby

    return pd.DataFrame(match_round_records)
"""
def team_side_assignment(
        match_df: pd.DataFrame,
        round_number: int
        ) -> pd.DataFrame:
    """
    Assign team sides (Attackers/Defenders) for each match
    """
    if round_number == 1:
        side_assignment = np.random.choice([0,1])
        match_df["isAttacker"] = match_df.apply(
            lambda x: side_assignment if x["team A"] == 1 and side_assignment==1 else 0, axis=1
            )
        match_df["isDefender"] = 1 - match_df["isAttacker"]
    elif round_number == 13:
        match_df["isAttacker"] = 1 - match_df["isAttacker"]
        match_df["isDefender"] = 1 - match_df["isAttacker"]
    return match_df

In [7]:
round_df = generating_full_match_details_per_round(
    match_df=df,
    agents_df=agents_df
)
round_df.head()

,match_id,match_date,user_id,agent_id,agent_name,map_id,map_name,join_date,username,tagline,rank_tier_uuid,team A,team B,isAttacker,isDefender,round_id
0,MATCH_000001,2025-01-05,840,efba5359-4016-a1e5-7626-b1ae76895940,Vyse,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0840,#4199,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R01
1,MATCH_000001,2025-01-05,814,0e38b510-41a8-5780-5e8f-568b2a4f2d6c,Iso,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-02,player0814,#2999,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R01
2,MATCH_000001,2025-01-05,215,95b78ed7-4637-86d9-7e41-71ba8c293152,Harbor,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0215,#7345,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R01
3,MATCH_000001,2025-01-05,101,add6443a-41bd-e414-f6ad-e58d267f4e95,Jett,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-01,player0101,#6371,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01
4,MATCH_000001,2025-01-05,920,1e58de9c-4950-5125-93e9-a0aee9f98746,Killjoy,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-01,player0920,#5618,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01


In [35]:
round1 = round_df[(round_df["match_id"]=="MATCH_000001") & (round_df["round_id"]=="MATCH_000001-R01")]
round2 = round_df[(round_df["match_id"]=="MATCH_000001") & (round_df["round_id"]=="MATCH_000001-R02")]

In [30]:
def events_per_round(
        round_df: pd.DataFrame,
        attackers_alive: int = 5,
        defenders_alive: int = 5,
        ) -> Dict[str, Any]:
    """
    For each round in match_df, generate event details:
        - kills, deaths, plants, defuses, etc.
    """
    records = []
    
    attacker_team = round_df[round_df["isAttacker"]==1]["agent_name"].tolist()
    defender_team = round_df[round_df["isDefender"]==1]["agent_name"].tolist()

    dead_attackers = []
    dead_defenders = []

    round_timer_expired = None
    total_duration_round = None
    team_spike_planted = None
    team_spike_diffused = None
    team_spike_detonated = None


    for row in round_df.itertuples(index=False):
        row_dict = dict(zip(round_df.columns, row))

        #simulate kills, deaths
        if row_dict["agent_name"] in attacker_team or row_dict["agent_name"] in defender_team:
            death = 0
            kill= 0
            if row_dict["isAttacker"] == 1 and row_dict["agent_name"] in attacker_team:
                kill = np.random.randint(0,defenders_alive+1) if row_dict["agent_name"] in attacker_team and defenders_alive > 0 else 0
                death = np.random.randint(0,2) if row_dict["agent_name"] in attacker_team and row_dict["agent_name"] not in dead_attackers else 1
                if defenders_alive > 0 and kill > 0:
                    defenders_alive -= kill
                    killed_agents = np.random.choice(defender_team, size=kill, replace=False)
                    dead_defenders = [agent for agent in defender_team if agent in killed_agents]
                attackers_alive = attackers_alive - 1 if death==1 else attackers_alive
                dead_attackers = [agent for agent in attacker_team if agent == row_dict["agent_name"]] if death==1 else dead_attackers
                spike_planted = np.random.choice([0,1], p=[0.3,0.7]) if team_spike_planted != 1 else 0
                row_dict["plants"] = spike_planted
                team_spike_planted = spike_planted if team_spike_planted is None or team_spike_planted == 0 else team_spike_planted

            elif row_dict["isDefender"] == 1 and row_dict["agent_name"] in defender_team:
                kill = np.random.randint(0,attackers_alive+1) if row_dict["agent_name"] in defender_team and attackers_alive > 0 else 0
                death = np.random.randint(0,2) if row_dict["agent_name"] in defender_team and row_dict["agent_name"] not in dead_defenders else 1
                if attackers_alive > 0 and kill > 0:
                    attackers_alive -= kill
                    killed_agents = np.random.choice(attacker_team, size=kill, replace=False)
                    dead_attackers = [agent for agent in attacker_team if agent in killed_agents]
                defenders_alive = defenders_alive - 1 if death==1 else defenders_alive
                dead_defenders = [agent for agent in defender_team if agent == row_dict["agent_name"]] if death==1 else dead_defenders
                spike_diffused = np.random.choice([0,1], p=[0.8,0.2]) if team_spike_planted == 1 and (team_spike_diffused is None or team_spike_diffused == 0) else 0
                row_dict["defussed"] = spike_diffused
                team_spike_diffused = spike_diffused if team_spike_diffused is None or team_spike_diffused == 0 else team_spike_diffused

            row_dict["kills"] = kill
            row_dict["death"] = death
        else:
            row_dict["kills"] = 0
            row_dict["death"] = 1
            row_dict["plants"] = 0
            row_dict["defussed"] = 0
        
        records.append(row_dict)


    if team_spike_planted == 1:
        if team_spike_diffused == 1:
            attacker_round_win = 0
            defender_round_win = 1
        else:
            attacker_round_win = 1
            defender_round_win = 0
    else: # no spike planted
        if attackers_alive > defenders_alive:
            attacker_round_win = 1
            defender_round_win = 0
        else:
            attacker_round_win = 0
            defender_round_win = 1
    
                
    return records, team_spike_planted, team_spike_diffused, attacker_round_win, defender_round_win

In [49]:
attacker_dict = {agent:250 for agent in round1[round1["isAttacker"]==1]["agent_name"].tolist()}
defender_dict = {agent:250 for agent in round1[round1["isDefender"]==1]["agent_name"].tolist()}

def kill_death_simulation_by_damage(
        row_dict: Dict[str, Any],
        attacker_dict: Dict[str, int],
        defender_dict: Dict[str, int],
)->Dict[str, Any]:
    
    kill = 0
    death = 0
    if row_dict["isAttacker"] == 1:
        for agent,health in defender_dict.items():
            attacker_agent_health = attacker_dict.get(row_dict["agent_name"])
            hit_value = np.random.randint(0,health+1)
            damage_value = np.random.randint(0,attacker_agent_health+1)
            if damage_value >= attacker_agent_health:
                attacker_dict[row_dict["agent_name"]] = 0
                death = 1
                break
            else:
                attacker_dict[row_dict["agent_name"]] = attacker_agent_health - damage_value

            if hit_value >= health:
                defender_dict[agent] = 0
                kill += 1
            else:
                defender_dict[agent] = health - hit_value

    elif row_dict["isDefender"] == 1:
        for agent,health in attacker_dict.items():
            defender_agent_health = defender_dict.get(row_dict["agent_name"])
            hit_value = np.random.randint(0,health+1)
            damage_value = np.random.randint(0,defender_agent_health)+1
            if damage_value >= defender_agent_health:
                defender_dict[row_dict["agent_name"]] = 0
                death = 1
                break
            else:
                defender_dict[row_dict["agent_name"]] = defender_agent_health - damage_value

            if hit_value >= health:
                attacker_dict[agent] = 0
                kill += 1
            else:
                attacker_dict[agent] = health - hit_value
    row_dict["kills"] = kill
    row_dict["death"] = death


    return row_dict, attacker_dict, defender_dict

In [50]:
records = []
for row in round1.itertuples(index=False):
        row_dict = dict(zip(round1.columns, row))
        row_dict, attacker_dict, defender_dict = kill_death_simulation_by_damage(
            row_dict=row_dict,
            attacker_dict=attacker_dict,
            defender_dict=defender_dict
        )
        records.append(row_dict)

In [51]:
r = pd.DataFrame(records)
r

,match_id,match_date,user_id,agent_id,agent_name,map_id,map_name,join_date,username,tagline,rank_tier_uuid,team A,team B,isAttacker,isDefender,round_id,kills,death
0,MATCH_000001,2025-01-05,840,efba5359-4016-a1e5-7626-b1ae76895940,Vyse,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0840,#4199,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R01,0,0
1,MATCH_000001,2025-01-05,814,0e38b510-41a8-5780-5e8f-568b2a4f2d6c,Iso,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-02,player0814,#2999,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R01,0,1
2,MATCH_000001,2025-01-05,215,95b78ed7-4637-86d9-7e41-71ba8c293152,Harbor,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0215,#7345,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R01,0,1
3,MATCH_000001,2025-01-05,101,add6443a-41bd-e414-f6ad-e58d267f4e95,Jett,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-01,player0101,#6371,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01,2,0
4,MATCH_000001,2025-01-05,920,1e58de9c-4950-5125-93e9-a0aee9f98746,Killjoy,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-01,player0920,#5618,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01,0,1
5,MATCH_000001,2025-01-05,258,a3bfb853-43b2-7238-a4f1-ad90e9e46bcc,Reyna,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0258,#0756,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01,2,0
6,MATCH_000001,2025-01-05,116,7f94d92c-4234-0a36-9646-3a87eb8b5c89,Yoru,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0116,#1207,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01,2,0
7,MATCH_000001,2025-01-05,948,6f2a04ca-43e0-be17-7f36-b3908627744d,Skye,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0948,#0983,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R01,1,1
8,MATCH_000001,2025-01-05,594,601dbbe7-43ce-be57-2a40-4abd24953621,KAY/O,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0594,#9892,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01,4,0
9,MATCH_000001,2025-01-05,100,320b2a48-4d9b-a075-30f1-1f93a9b638fa,Sova,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-02,player0100,#0874,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R01,0,1


In [52]:
r[r["isAttacker"]==1]

,match_id,match_date,user_id,agent_id,agent_name,map_id,map_name,join_date,username,tagline,rank_tier_uuid,team A,team B,isAttacker,isDefender,round_id,kills,death
3,MATCH_000001,2025-01-05,101,add6443a-41bd-e414-f6ad-e58d267f4e95,Jett,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-01,player0101,#6371,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01,2,0
4,MATCH_000001,2025-01-05,920,1e58de9c-4950-5125-93e9-a0aee9f98746,Killjoy,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-01,player0920,#5618,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01,0,1
5,MATCH_000001,2025-01-05,258,a3bfb853-43b2-7238-a4f1-ad90e9e46bcc,Reyna,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0258,#0756,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01,2,0
6,MATCH_000001,2025-01-05,116,7f94d92c-4234-0a36-9646-3a87eb8b5c89,Yoru,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0116,#1207,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01,2,0
8,MATCH_000001,2025-01-05,594,601dbbe7-43ce-be57-2a40-4abd24953621,KAY/O,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0594,#9892,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01,4,0


In [31]:
round_details, team_spike_planted, team_spike_diffused, attacker_round_win, defender_round_win = events_per_round(round1)
round_details  = pd.DataFrame(round_details)
round_details     

,match_id,match_date,user_id,agent_id,agent_name,map_id,map_name,join_date,username,tagline,rank_tier_uuid,team A,team B,isAttacker,isDefender,round_id,defussed,kills,death,plants
0,MATCH_000001,2025-01-05,840,efba5359-4016-a1e5-7626-b1ae76895940,Vyse,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0840,#4199,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R01,0.0,4,0,NaN
1,MATCH_000001,2025-01-05,814,0e38b510-41a8-5780-5e8f-568b2a4f2d6c,Iso,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-02,player0814,#2999,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R01,0.0,0,0,NaN
2,MATCH_000001,2025-01-05,215,95b78ed7-4637-86d9-7e41-71ba8c293152,Harbor,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0215,#7345,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R01,0.0,0,1,NaN
3,MATCH_000001,2025-01-05,101,add6443a-41bd-e414-f6ad-e58d267f4e95,Jett,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-01,player0101,#6371,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01,NaN,0,1,0.0
4,MATCH_000001,2025-01-05,920,1e58de9c-4950-5125-93e9-a0aee9f98746,Killjoy,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-01,player0920,#5618,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01,NaN,1,1,1.0
5,MATCH_000001,2025-01-05,258,a3bfb853-43b2-7238-a4f1-ad90e9e46bcc,Reyna,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0258,#0756,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01,NaN,0,0,0.0
6,MATCH_000001,2025-01-05,116,7f94d92c-4234-0a36-9646-3a87eb8b5c89,Yoru,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0116,#1207,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01,NaN,1,1,0.0
7,MATCH_000001,2025-01-05,948,6f2a04ca-43e0-be17-7f36-b3908627744d,Skye,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0948,#0983,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R01,0.0,0,1,NaN
8,MATCH_000001,2025-01-05,594,601dbbe7-43ce-be57-2a40-4abd24953621,KAY/O,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0594,#9892,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01,NaN,1,1,0.0
9,MATCH_000001,2025-01-05,100,320b2a48-4d9b-a075-30f1-1f93a9b638fa,Sova,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-02,player0100,#0874,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R01,0.0,0,1,NaN


In [33]:
round_details[round_details["isAttacker"] == 1]

,match_id,match_date,user_id,agent_id,agent_name,map_id,map_name,join_date,username,tagline,rank_tier_uuid,team A,team B,isAttacker,isDefender,round_id,defussed,kills,death,plants
3,MATCH_000001,2025-01-05,101,add6443a-41bd-e414-f6ad-e58d267f4e95,Jett,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-01,player0101,#6371,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01,NaN,0,1,0.0
4,MATCH_000001,2025-01-05,920,1e58de9c-4950-5125-93e9-a0aee9f98746,Killjoy,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-01,player0920,#5618,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01,NaN,1,1,1.0
5,MATCH_000001,2025-01-05,258,a3bfb853-43b2-7238-a4f1-ad90e9e46bcc,Reyna,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0258,#0756,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01,NaN,0,0,0.0
6,MATCH_000001,2025-01-05,116,7f94d92c-4234-0a36-9646-3a87eb8b5c89,Yoru,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0116,#1207,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01,NaN,1,1,0.0
8,MATCH_000001,2025-01-05,594,601dbbe7-43ce-be57-2a40-4abd24953621,KAY/O,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0594,#9892,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R01,NaN,1,1,0.0


In [34]:
round_details[round_details["isDefender"] == 1]

,match_id,match_date,user_id,agent_id,agent_name,map_id,map_name,join_date,username,tagline,rank_tier_uuid,team A,team B,isAttacker,isDefender,round_id,defussed,kills,death,plants
0,MATCH_000001,2025-01-05,840,efba5359-4016-a1e5-7626-b1ae76895940,Vyse,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0840,#4199,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R01,0.0,4,0,NaN
1,MATCH_000001,2025-01-05,814,0e38b510-41a8-5780-5e8f-568b2a4f2d6c,Iso,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-02,player0814,#2999,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R01,0.0,0,0,NaN
2,MATCH_000001,2025-01-05,215,95b78ed7-4637-86d9-7e41-71ba8c293152,Harbor,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0215,#7345,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R01,0.0,0,1,NaN
7,MATCH_000001,2025-01-05,948,6f2a04ca-43e0-be17-7f36-b3908627744d,Skye,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0948,#0983,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R01,0.0,0,1,NaN
9,MATCH_000001,2025-01-05,100,320b2a48-4d9b-a075-30f1-1f93a9b638fa,Sova,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-02,player0100,#0874,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R01,0.0,0,1,NaN


In [36]:
round_details_2, team_spike_planted, team_spike_diffused, attacker_round_win, defender_round_win = events_per_round(round2)
round_details_2  = pd.DataFrame(round_details_2)
round_details_2     

,match_id,match_date,user_id,agent_id,agent_name,map_id,map_name,join_date,username,tagline,rank_tier_uuid,team A,team B,isAttacker,isDefender,round_id,defussed,kills,death,plants
0,MATCH_000001,2025-01-05,840,efba5359-4016-a1e5-7626-b1ae76895940,Vyse,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0840,#4199,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R02,0.0,2,1,NaN
1,MATCH_000001,2025-01-05,814,0e38b510-41a8-5780-5e8f-568b2a4f2d6c,Iso,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-02,player0814,#2999,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R02,0.0,0,0,NaN
2,MATCH_000001,2025-01-05,215,95b78ed7-4637-86d9-7e41-71ba8c293152,Harbor,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0215,#7345,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R02,0.0,0,1,NaN
3,MATCH_000001,2025-01-05,101,add6443a-41bd-e414-f6ad-e58d267f4e95,Jett,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-01,player0101,#6371,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R02,NaN,2,1,0.0
4,MATCH_000001,2025-01-05,920,1e58de9c-4950-5125-93e9-a0aee9f98746,Killjoy,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-01,player0920,#5618,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R02,NaN,1,1,1.0
5,MATCH_000001,2025-01-05,258,a3bfb853-43b2-7238-a4f1-ad90e9e46bcc,Reyna,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0258,#0756,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R02,NaN,0,1,0.0
6,MATCH_000001,2025-01-05,116,7f94d92c-4234-0a36-9646-3a87eb8b5c89,Yoru,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0116,#1207,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R02,NaN,0,1,0.0
7,MATCH_000001,2025-01-05,948,6f2a04ca-43e0-be17-7f36-b3908627744d,Skye,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0948,#0983,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R02,0.0,0,1,NaN
8,MATCH_000001,2025-01-05,594,601dbbe7-43ce-be57-2a40-4abd24953621,KAY/O,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0594,#9892,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R02,NaN,0,1,0.0
9,MATCH_000001,2025-01-05,100,320b2a48-4d9b-a075-30f1-1f93a9b638fa,Sova,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-02,player0100,#0874,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R02,0.0,0,1,NaN


In [37]:
round_details_2[round_details_2["isAttacker"] == 1]

,match_id,match_date,user_id,agent_id,agent_name,map_id,map_name,join_date,username,tagline,rank_tier_uuid,team A,team B,isAttacker,isDefender,round_id,defussed,kills,death,plants
3,MATCH_000001,2025-01-05,101,add6443a-41bd-e414-f6ad-e58d267f4e95,Jett,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-01,player0101,#6371,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R02,NaN,2,1,0.0
4,MATCH_000001,2025-01-05,920,1e58de9c-4950-5125-93e9-a0aee9f98746,Killjoy,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-01,player0920,#5618,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R02,NaN,1,1,1.0
5,MATCH_000001,2025-01-05,258,a3bfb853-43b2-7238-a4f1-ad90e9e46bcc,Reyna,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0258,#0756,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R02,NaN,0,1,0.0
6,MATCH_000001,2025-01-05,116,7f94d92c-4234-0a36-9646-3a87eb8b5c89,Yoru,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0116,#1207,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R02,NaN,0,1,0.0
8,MATCH_000001,2025-01-05,594,601dbbe7-43ce-be57-2a40-4abd24953621,KAY/O,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0594,#9892,03621f52-342b-cf4e-4f86-9350a49c6d04,1,0,1,0,MATCH_000001-R02,NaN,0,1,0.0


In [38]:
round_details_2[round_details_2["isDefender"] == 1]

,match_id,match_date,user_id,agent_id,agent_name,map_id,map_name,join_date,username,tagline,rank_tier_uuid,team A,team B,isAttacker,isDefender,round_id,defussed,kills,death,plants
0,MATCH_000001,2025-01-05,840,efba5359-4016-a1e5-7626-b1ae76895940,Vyse,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-05,player0840,#4199,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R02,0.0,2,1,NaN
1,MATCH_000001,2025-01-05,814,0e38b510-41a8-5780-5e8f-568b2a4f2d6c,Iso,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-02,player0814,#2999,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R02,0.0,0,0,NaN
2,MATCH_000001,2025-01-05,215,95b78ed7-4637-86d9-7e41-71ba8c293152,Harbor,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0215,#7345,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R02,0.0,0,1,NaN
7,MATCH_000001,2025-01-05,948,6f2a04ca-43e0-be17-7f36-b3908627744d,Skye,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-03,player0948,#0983,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R02,0.0,0,1,NaN
9,MATCH_000001,2025-01-05,100,320b2a48-4d9b-a075-30f1-1f93a9b638fa,Sova,12452a9d-48c3-0b02-e7eb-0381c3520404,Kasbah,2025-01-02,player0100,#0874,03621f52-342b-cf4e-4f86-9350a49c6d04,0,1,0,1,MATCH_000001-R02,0.0,0,1,NaN


In [39]:
attacker_list_temp = ["Vyse", "Jett", "Phoenix", "Raze", "Reyna"]
attacker_dict = {agent:250 for agent in attacker_list_temp}
attacker_dict

{'Vyse': 250, 'Jett': 250, 'Phoenix': 250, 'Raze': 250, 'Reyna': 250}

In [40]:
row_dict = {"agent":"Jett"}

attacker_dict.get(row_dict["agent"])

250

In [41]:
attacker_dict["Jett"] = 150

In [42]:
attacker_dict.get(row_dict["agent"])

150

In [43]:
attacker_dict

{'Vyse': 250, 'Jett': 150, 'Phoenix': 250, 'Raze': 250, 'Reyna': 250}

In [8]:
import pandas as pd

In [39]:
temp_list = {"agents": ["Vyse", "Jett", "Phoenix", "Raze", "Reyna", "Vyse", "Jett", "Phoenix", "Raze", "Reyna"],
             "health": [250, 250, 250, 250, 250, 150, 150, 150, 150, 150],
             "damage": [100,100,100,100,100,100,100,100,100,100]
            }

temp_df = pd.DataFrame(temp_list)
temp_df.drop(columns=["damage"],inplace=True)
temp_df.columns


Index(['agents', 'health'], dtype='str')

In [14]:
temp = temp_df[["agents","damage"]].drop_duplicates()
temp

,agents,damage
0,Vyse,100
1,Jett,100
2,Phoenix,100
3,Raze,100
4,Reyna,100


In [7]:
temp

In [15]:
import numpy as np

np.random.rand(3)

array([0.07021989, 0.50568932, 0.54417804])

In [16]:
import numpy as np

def divide_number_randomly(total_number):
    # Generate 3 random numbers
    random_parts = np.random.rand(3)
    # Normalize them to sum to 1
    normalized_parts = random_parts / sum(random_parts)
    # Multiply by the total number
    result = normalized_parts * total_number
    return result

# Example: Divide 100 into 3 parts
num = 100
parts = divide_number_randomly(num)
print(f"Parts: {parts}")
print(f"Sum: {sum(parts)}")

Parts: [16.87608874 79.24172117  3.88219008]
Sum: 100.00000000000001


In [1]:
import sys
print(sys.path)

['c:\\Users\\gjric\\Desktop\\ValorantAnalysis\\valorant_env\\python313.zip', 'c:\\Users\\gjric\\Desktop\\ValorantAnalysis\\valorant_env\\DLLs', 'c:\\Users\\gjric\\Desktop\\ValorantAnalysis\\valorant_env\\Lib', 'c:\\Users\\gjric\\Desktop\\ValorantAnalysis\\valorant_env', '', 'c:\\Users\\gjric\\Desktop\\ValorantAnalysis\\valorant_env\\Lib\\site-packages', 'c:\\Users\\gjric\\Desktop\\ValorantAnalysis\\valorant_env\\Lib\\site-packages\\win32', 'c:\\Users\\gjric\\Desktop\\ValorantAnalysis\\valorant_env\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\gjric\\Desktop\\ValorantAnalysis\\valorant_env\\Lib\\site-packages\\Pythonwin']


In [2]:
import numpy as np

def biased_hbl_percentages(random_state=None):
    """
    Return (head_pct, body_pct, leg_pct) that sum to 100,
    biased so head > body > leg on average.
    """
    if random_state is None:
        random_state = np.random.RandomState()

    # Dirichlet parameters control the mean:
    # larger alpha -> more stable around that mean.
    # Example: mean ≈ [0.5, 0.35, 0.15]
    alpha = np.array([5.0, 3.5, 1.5])
    sample = random_state.dirichlet(alpha)  # sums to 1

    head_pct = round(sample[0] * 100, 2)
    body_pct = round(sample[1] * 100, 2)
    leg_pct  = round(100 - head_pct - body_pct, 2)

    return head_pct, body_pct, leg_pct


In [3]:
head_pct, body_pct, leg_pct = biased_hbl_percentages()
head_damage = 180 * head_pct / 100
body_damage = 180 * body_pct / 100
leg_damage  = 180 * leg_pct / 100
print(f"Headshot %: {head_pct}%, Damage: {head_damage}")
print(f"Bodyshot %: {body_pct}%, Damage: {body_damage}")
print(f"Legshot %: {leg_pct}%, Damage: {leg_damage}")

Headshot %: 47.14%, Damage: 84.852
Bodyshot %: 48.49%, Damage: 87.28200000000001
Legshot %: 4.37%, Damage: 7.8660000000000005


In [23]:
attacker_list = ["Vyse", "Jett", "Phoenix", "Raze", "Reyna"]
defender_list = ["Sage", "Brimstone", "Cypher", "Killjoy", "Omen"]

attackers_shot_pct = {agent:{def_agent:{"hit": {"head":0, "body":0, "leg":0},"damage":{"head":0,"body":0,"leg":0}} for def_agent in defender_list} for agent in attacker_list}
defenders_shot_pct = {agent:{att_agent:{"hit": {"head":0, "body":0, "leg":0},"damage":{"head":0,"body":0,"leg":0}} for att_agent in attacker_list} for agent in defender_list}
    

In [35]:
agents_hit_damage = {agent:{ag: {"hit": {"head":0, "body":0, "leg":0},"damage":{"head":0,"body":0,"leg":0},"outgoing_damage":0, "incoming_damage":0} for ag in attacker_list+defender_list } for agent in attacker_list+defender_list}

In [36]:
agents_hit_damage

{'Vyse': {'Vyse': {'hit': {'head': 0, 'body': 0, 'leg': 0},
   'damage': {'head': 0, 'body': 0, 'leg': 0},
   'outgoing_damage': 0,
   'incoming_damage': 0},
  'Jett': {'hit': {'head': 0, 'body': 0, 'leg': 0},
   'damage': {'head': 0, 'body': 0, 'leg': 0},
   'outgoing_damage': 0,
   'incoming_damage': 0},
  'Phoenix': {'hit': {'head': 0, 'body': 0, 'leg': 0},
   'damage': {'head': 0, 'body': 0, 'leg': 0},
   'outgoing_damage': 0,
   'incoming_damage': 0},
  'Raze': {'hit': {'head': 0, 'body': 0, 'leg': 0},
   'damage': {'head': 0, 'body': 0, 'leg': 0},
   'outgoing_damage': 0,
   'incoming_damage': 0},
  'Reyna': {'hit': {'head': 0, 'body': 0, 'leg': 0},
   'damage': {'head': 0, 'body': 0, 'leg': 0},
   'outgoing_damage': 0,
   'incoming_damage': 0},
  'Sage': {'hit': {'head': 0, 'body': 0, 'leg': 0},
   'damage': {'head': 0, 'body': 0, 'leg': 0},
   'outgoing_damage': 0,
   'incoming_damage': 0},
  'Brimstone': {'hit': {'head': 0, 'body': 0, 'leg': 0},
   'damage': {'head': 0, 'body'

In [27]:
print(agents)

['Vyse', 'Jett', 'Phoenix', 'Raze', 'Reyna', 'Sage', 'Brimstone', 'Cypher', 'Killjoy', 'Omen']


In [28]:
agents_df = pd.DataFrame(agents)

KeyError: 'Vyse'

In [7]:
df

,Vyse.Sage.hit.head,Vyse.Sage.hit.body,Vyse.Sage.hit.leg,Vyse.Sage.damage.head,Vyse.Sage.damage.body,Vyse.Sage.damage.leg,Vyse.Brimstone.hit.head,Vyse.Brimstone.hit.body,Vyse.Brimstone.hit.leg,Vyse.Brimstone.damage.head,...,Reyna.Killjoy.hit.leg,Reyna.Killjoy.damage.head,Reyna.Killjoy.damage.body,Reyna.Killjoy.damage.leg,Reyna.Omen.hit.head,Reyna.Omen.hit.body,Reyna.Omen.hit.leg,Reyna.Omen.damage.head,Reyna.Omen.damage.body,Reyna.Omen.damage.leg
0,100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
